***
### Consecutive Prime Sum
***
Solution to:
https://projecteuler.net/problem=50

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
#Obtain prime numbers up to N
# Using Sieve the Eratosthenes: O(NloglogN)

# Algorithm implemented with pandas
def get_primes_pandas(N):
    df = pd.DataFrame({'IsPrime':np.ones(N)})
    df.IsPrime = df.IsPrime.astype(np.int8)
    df.drop([0,1],inplace=True)
    
    primes = []
    check = 2
    while check*check<=N:
        df = df.loc[df['IsPrime']==1,:]
        check = df.index[0]

        primes.append(check)
        df.loc[df.index%check==0]=0
      
    # Contains remaining primes, from the second element
    indexes = list(df.index)
    del df
    return primes+indexes[1:]

In [3]:
# Same algorithm, implemented with lists
def get_primes(N):
    IsPrime = [1 for i in range(N+1)]
    
    primes = []
    check = 2
    while check*check<=N:
        if IsPrime[check]:
            primes.append(check)
            # Increment by check
            for i in range(check*check,N+1,check):
                IsPrime[i]=0
        check+=1

    # Store all remaining primes in an array
    for num in range(check,N):
        if IsPrime[num]:
            primes.append(num)
    del IsPrime
    return primes

In [4]:
# Variable
N=1000000

In [5]:
%time primes_pandas = get_primes_pandas(N)

CPU times: user 656 ms, sys: 33.6 ms, total: 690 ms
Wall time: 694 ms


In [6]:
%time primes = get_primes(N)

CPU times: user 232 ms, sys: 6.53 ms, total: 239 ms
Wall time: 238 ms


#### Using faster method:
 get_primes

In [7]:
def sum_consecutive(prime_list,N):

    primes = prime_list.copy()
    # Longest sequence added that is < N
    # Sum of first X primes >> Sum of first X even numbers = N(N+1)
    
    max_slide = int(np.round(N**.5))
    # A sum of sequence that starts with 2, can only have a even length 
    # to produce an odd number (possibly prime)
    max_slide -= max_slide%2   
    
    max_sum = np.sum(primes[:max_slide])
    first_prime = primes[0]
         
    # Get the max sum (prime or not) that is <= the highest prime < N
    while max_sum >= primes[-1]:
        max_slide-=2
        max_sum-=(primes[max_slide]+primes[max_slide-1])
    
    # Sequence starting with first element: 2
    # Get the max sum that is a prime, with a sequence starting with 2
    while max_sum not in primes:
        max_slide -= 2
        max_sum = np.sum(primes[:max_slide])
        
    # Check other primes
    check = True

    # Save max slide when sequence starts with 2
    max_slide_even_prime = max_slide
    
    # Check for sequences starting with other primes
    max_slide+=1
    
    while check:
        primes.pop(0) #Remove first element

        # Check if must continue checking
        if np.sum(primes[:max_slide])>primes[-1]:
            check = False
        
        slide, sum_slide, first_element = find_slide(max_slide,primes)
        
        # Update longest slide and its sum
        if slide>max_slide:
            max_slide = slide
            if sum_slide:
                max_sum = sum_slide
                first_prime = first_element

    if first_prime==2:
        return max_slide_even_prime, max_sum, first_prime
    else:
        return max_slide, max_sum, first_prime


def find_slide(slide,primes):
    
    # Check only on any longer sequences
    # Increase slide by 2 since for a sum of even amount of prime numbers
    # will result in an even number (not prime)
    sum_prime = np.sum(primes[:slide+2])
    
    max_sum = None
    max_slide = slide
    first_element = None
    

    #Only check while sum < max # prime
    while sum_prime<=primes[-1]:         
        slide+=2
        # Update variables if necessary
        if sum_prime in primes:
            max_slide = slide
            max_sum = sum_prime
            first_element = primes[0]
        
        sum_prime = np.sum(primes[:slide+2])
      
    return max_slide, max_sum, first_element

In [8]:
# Max_slide refers to the longest sequence found
%time max_slide, prime_sum, first = sum_consecutive(primes,N)

CPU times: user 174 ms, sys: 2.02 ms, total: 176 ms
Wall time: 174 ms


In [9]:
print('The longest sequence contains {0} primes, and its sum/prime is {1}'.format(max_slide,prime_sum))
print('Sequence starts with prime: ',first)

The longest sequence contains 543 primes, and its sum/prime is 997651
Sequence starts with prime:  7


***Prime 7 is at position primes[3]***

[2,3,5,7,11,...]

     ^
     |

Double checking the sequence sum is a prime and matches with previous result

In [10]:
index = primes.index(first)
max_prime_sum = np.sum(primes[index:index+max_slide])

if max_prime_sum in primes and max_prime_sum==prime_sum:
    print('{} is a prime!'.format(prime_sum))
else:
    print('{} is not a prime!'.format(prime_sum))

997651 is a prime!
